<a href="https://colab.research.google.com/github/gabriquaranta/drones-stuff/blob/main/yolo5_torch_hub_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download dasmehdixtr/drone-dataset-uav

!mkdir ./datasets
!mkdir ./datasets/train
!mkdir ./datasets/val
!mkdir ./datasets/test

!unzip -q drone-dataset-uav.zip
!rm -rf ./dataset_xml_format/

Dataset URL: https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav
License(s): copyright-authors
100% 716M/716M [00:08<00:00, 88.6MB/s]
100% 716M/716M [00:08<00:00, 84.8MB/s]


In [3]:
import os
import random

In [2]:
dirname='./drone_dataset_yolo/dataset_txt'

#count txt files in drone_dataset_yoy/dataset_txt
filenames = [f[:-4] for f in os.listdir(dirname) if f.endswith('.txt')]
n_pics=len(filenames)
print(n_pics)

ids=[i for i in range(n_pics)]
random.shuffle(ids)
#print(ids)

trainratio=0.8
valratio=0.1
testratio=0.1

train_ids=ids[:int(n_pics*trainratio)]
val_ids=ids[int(n_pics*trainratio):int(n_pics*(trainratio+valratio))]
test_ids=ids[int(n_pics*(trainratio+valratio)):]

print(len(train_ids), len(val_ids), len(test_ids))

# cp images to corresponding train or test folders
for i in train_ids:
  try:
    !cp ./drone_dataset_yolo/dataset_txt/{filenames[i]}.txt ./datasets/train/{filenames[i]}.txt
    !cp ./drone_dataset_yolo/dataset_txt/{filenames[i]}.jpg ./datasets/train/{filenames[i]}.jpg
  except:
    print(filenames[i])
    continue

for i in val_ids:
  try:
    !cp ./drone_dataset_yolo/dataset_txt/{filenames[i]}.txt ./datasets/val/{filenames[i]}.txt
    !cp ./drone_dataset_yolo/dataset_txt/{filenames[i]}.jpg ./datasets/val/{filenames[i]}.jpg
  except:
    print(filenames[i])
    continue

for i in test_ids:
  try:
    !cp ./drone_dataset_yolo/dataset_txt/{filenames[i]}.txt ./datasets/test/{filenames[i]}.txt
    !cp ./drone_dataset_yolo/dataset_txt/{filenames[i]}.jpg ./datasets/test/{filenames[i]}.jpg
  except:
    print(filenames[i])
    continue

1360
1088 136 136
cp: cannot stat './drone_dataset_yolo/dataset_txt/classes.jpg': No such file or directory


In [4]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-1 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [12]:
path='./datasets/test'
imgs = [os.path.join(path,i) for i in os.listdir(path) if i.endswith('.jpg')]
print(len(imgs))
results = model(imgs)
results.print()

136


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/136: 183x275 1 airplane
image 2/136: 424x636 1 airplane
image 3/136: 400x710 1 airplane
image 4/136: 800x1200 1 airplane
image 5/136: 720x1280 1 airplane
image 6/136: 720x1280 (no detections)
image 7/136: 720x1280 1 airplane
image 8/136: 800x800 (no detections)
image 9/136: 810x1440 1 bicycle
image 10/136: 418x740 1 motorcycle
image 11/136: 720x1280 1 airplane
image 12/136: 456x810 1 airplane
image 13/136: 643x1060 (no detections)
image 14/136: 355x642 1 airplane
image 15/136: 390x652 1 airplane
image 16/136: 720x1280 1 car, 1 airplane
image 17/136: 312x500 1 airplane
image 18/136: 720x1280 1 airplane
image 19/136: 1088x1920 1 person
image 20/136: 550x900 4 airplanes
image 21/136: 720x1280 1 person
image 22/136: 3750x5625 1 airplane
image 23/136: 361x525 1 